In [182]:
from qutip import*
import numpy as np
import matplotlib.pyplot as plt

In [183]:
steps = 100 #size of the step (precision of simulation)
utau = 0.5 #size of the gap
delta= 1 #detuning
u = 0.0001 #sweep rate
tau = utau/u
H0 = delta*sigmaz() #time-independent term, includes the value of delta
H1 = sigmax() #time-dependent term

In [184]:
def f1_t(t, args):
    """ Returns the value of the driving function.
        Must receive a sweep rate u.                """
    return  t * args["sweep_rate"] # Linear control (Landau-Zener -> u t) 

In [185]:
tlist = np.linspace(-tau,0,steps)
print(tlist)

[-5000.         -4949.49494949 -4898.98989899 -4848.48484848
 -4797.97979798 -4747.47474747 -4696.96969697 -4646.46464646
 -4595.95959596 -4545.45454545 -4494.94949495 -4444.44444444
 -4393.93939394 -4343.43434343 -4292.92929293 -4242.42424242
 -4191.91919192 -4141.41414141 -4090.90909091 -4040.4040404
 -3989.8989899  -3939.39393939 -3888.88888889 -3838.38383838
 -3787.87878788 -3737.37373737 -3686.86868687 -3636.36363636
 -3585.85858586 -3535.35353535 -3484.84848485 -3434.34343434
 -3383.83838384 -3333.33333333 -3282.82828283 -3232.32323232
 -3181.81818182 -3131.31313131 -3080.80808081 -3030.3030303
 -2979.7979798  -2929.29292929 -2878.78787879 -2828.28282828
 -2777.77777778 -2727.27272727 -2676.76767677 -2626.26262626
 -2575.75757576 -2525.25252525 -2474.74747475 -2424.24242424
 -2373.73737374 -2323.23232323 -2272.72727273 -2222.22222222
 -2171.71717172 -2121.21212121 -2070.70707071 -2020.2020202
 -1969.6969697  -1919.19191919 -1868.68686869 -1818.18181818
 -1767.67676768 -1717.17171

In [186]:
Ht1 = -utau*H1+H0 #The Hamiltonian at time t_1=-tau
excited_t1 = Ht1.eigenstates()[1][1] #excited state at t_1 
Ht2 = H0 # Hamiltonian at time t_2=0
excited_t2 = Ht2.eigenstates()[1][1] #excited state at t_2



In [187]:
print(Ht2.eigenstates()[1][1])

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[-1.]
 [ 0.]]


In [188]:
Ht = QobjEvo([H0,[H1,f1_t]],{'sweep_rate':u}) #Define the object for the time dependent Hamiltonian

Obtaining the state vector

In [189]:

psi0 = excited_t1 #the inital state is the excited state at t_1 
result = sesolve(Ht,psi0,tlist)
final_state = result.states[-1] # equivalent to U|psi_0> at the end of the evolution

In [193]:
P = (excited_t2.dag()*final_state)*(final_state.dag()*excited_t2) #Permanence probability excited state
print(P)

Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[1.]]


In [194]:
print(final_state)
print(excited_t2)

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[-2.34738857e-01-9.72058469e-01j]
 [-4.03616406e-05+2.12697355e-06j]]
Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[-1.]
 [ 0.]]
